In [1]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # Carregando a ApiKey secreta

In [2]:
client = openai.Client()   # Instanciando a API client

In [3]:
mensagens = [{'role': 'user', 'content': 'O que é uma maçã em 5 palavras?'}]

# Capturando a resposta do modelo
resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)


### Acessando varios atributos da respota

resposta

resposta.id

resposta.choices...

### Imprimindo a resposta

In [4]:
resposta.choices[0].message.content

'Fruta redonda e saborosa.'

##### Se eu quero continuar a conversa, tenho que adicionar tambem a resposta do modelo

In [5]:
mensagens.append({'role': 'assistant', 'content': resposta.choices[0].message.content})

In [6]:
# Verificando todo o conteudo
mensagens

[{'role': 'user', 'content': 'O que é uma maçã em 5 palavras?'},
 {'role': 'assistant', 'content': 'Fruta redonda e saborosa.'}]

### Enviando proxima pergunta

In [7]:
mensagens.append({'role': 'user', 'content': 'E qual é a sua cor?'})

### Capturando e exibindo resposta

In [9]:
resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

In [10]:
resposta.choices[0].message.content

'Vermelha ou verde, dependendo da variedade.'

# Para isso, podemos criar uma função iterativa 


##### exclude_none e model_dump = Detalhes abaixo

In [11]:
# Definindo a função e passando como parametro as mensagens e os parametros padrao

def geracao_texto(mensagens, model='gpt-3.5-turbo-0125', max_tokens=1000, temperature=0):

    # Capturando a resposta do modelo
    resposta = client.chat.completions.create(   
        messages=mensagens,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature
    )
    
    print(resposta.choices[0].message.content)     # Imprimindo a resposta do modelo

    # Adicionado a resposta do modelo
    mensagens.append(resposta.choices[0].message.model_dump(exclude_none=True)) 
     

    return mensagens  # Retornando as mensagens

### Mandando as mensagens do usuario com a nova função e ja recebendo a resposta

In [12]:
mensagens = [{'role': 'user', 'content': 'O que é uma maçã em 5 palavras?'}]
mensagens = geracao_texto(mensagens)

Fruta redonda e saborosa.


### Continuando a conversa

In [13]:
mensagens.append({'role': 'user', 'content': 'E qual é a sua cor?'})
mensagens = geracao_texto(mensagens)

Vermelha, verde ou amarela.


# Explorando classe de resposta

##### Model_dump = Ja passa a resposta limpa, para evitar todos os itens descenessarios (detalhes abaixo)
##### Exclude_none = Excluir automaticamente campos None dos objetos JSON, resultando em um JSON limpo e conciso.


In [14]:

mensagens = [{'role': 'user', 'content': 'O que é uma maçã em 5 palavras?'}]
resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

In [15]:
# Resposta completa
resposta.choices[0].message

ChatCompletionMessage(content='Fruta redonda e saborosa.', role='assistant', function_call=None, tool_calls=None)

In [16]:
# Com model Dump
resposta.choices[0].message.model_dump()

{'content': 'Fruta redonda e saborosa.',
 'role': 'assistant',
 'function_call': None,
 'tool_calls': None}

In [17]:
# Model Dump e exclude_none
resposta.choices[0].message.model_dump(exclude_none=True)

{'content': 'Fruta redonda e saborosa.', 'role': 'assistant'}

In [18]:
# Uso de Tokens na resposta

resposta.usage

CompletionUsage(completion_tokens=9, prompt_tokens=20, total_tokens=29)

## Explorando max_tokens e temperature

In [19]:
# max_tokens = Limita o tamanho da resposta. Mas lembrando: Tokens != Palavras

mensagens = [{'role': 'user', 'content': 'O que é uma maçã em 5 palavras?'}]
mensagens = geracao_texto(mensagens, max_tokens=3)

Fruta red


In [20]:
# temperature = Quao aleatorio a resposta será? 0 é sempre a mesma resposta, 1 = Altera a resposta

mensagens = [{'role': 'user', 'content': 'O que é uma maçã?'}]
mensagens = geracao_texto(mensagens, temperature=1)

Uma maçã é o fruto da macieira, uma árvore pertencente à família Rosaceae. Possui uma casca geralmente vermelha ou verde, polpa branca e suculenta e sementes no seu interior. É uma fruta muito popular e amplamente consumida em todo o mundo, sendo conhecida por seu sabor adocicado e sua textura crocante. Além disso, é uma excelente fonte de vitaminas, fibras e antioxidantes, trazendo diversos benefícios para a saúde.


# Respondendo de forma gradativa

#### Notamos que ele dá toda a resposta completa, mas não é isso que ocorre em um chat, precisamos que ele vá respondendo aos poucos, como se tivesse conversando.

### Vamos enviar a pergunta e capturar a resposta

stream= Enviar resposta aos poucos

In [24]:
mensagens = [{'role': 'user', 'content': 'crie uma história sobre uma viagem a marte'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0,
    stream=True
)

### Imprimindo a resposta gradativamente

In [25]:
resposta_completa = ''
for stream_resposta in resposta:               # Iterando sobre cada parte da respota
    texto = stream_resposta.choices[0].delta.content  # Pegar so o texto, sem todos aqueles parametros
    if texto:
        resposta_completa += texto  # Se ainda houver texto, adicione na resposta completa
        print(texto, end='')  # end='' = Va escrevendo na mesma linha

Era o ano de 2050 e a humanidade finalmente havia conseguido enviar uma missão tripulada para Marte. A nave espacial, batizada de "Explorador Vermelho", estava pronta para decolar com uma equipe de astronautas altamente treinados e equipada com tecnologia de ponta.

A viagem até Marte durou meses, mas finalmente a nave chegou ao planeta vermelho. Os astronautas ficaram maravilhados com a paisagem árida e desolada, mas ao mesmo tempo fascinados com a possibilidade de explorar um novo mundo.

Eles desembarcaram na superfície de Marte e começaram a explorar, coletando amostras de solo e rochas para análise. Enquanto isso, a equipe na Terra monitorava a missão de perto, ansiosa para descobrir mais sobre o planeta vizinho.

Durante a exploração, os astronautas descobriram evidências de vida passada em Marte, o que causou grande comoção na comunidade científica. Eles também encontraram vestígios de antigas civilizações marcianas, o que levantou ainda mais questões sobre a história do planeta

## Imprimindo a resposta completa

In [26]:
print(resposta_completa)

Era o ano de 2050 e a humanidade finalmente havia conseguido enviar uma missão tripulada para Marte. A nave espacial, batizada de "Explorador Vermelho", estava pronta para decolar com uma equipe de astronautas altamente treinados e equipada com tecnologia de ponta.

A viagem até Marte durou meses, mas finalmente a nave chegou ao planeta vermelho. Os astronautas ficaram maravilhados com a paisagem árida e desolada, mas ao mesmo tempo fascinados com a possibilidade de explorar um novo mundo.

Eles desembarcaram na superfície de Marte e começaram a explorar, coletando amostras de solo e rochas para análise. Enquanto isso, a equipe na Terra monitorava a missão de perto, ansiosa para descobrir mais sobre o planeta vizinho.

Durante a exploração, os astronautas descobriram evidências de vida passada em Marte, o que causou grande comoção na comunidade científica. Eles também encontraram vestígios de antigas civilizações marcianas, o que levantou ainda mais questões sobre a história do planeta